In [8]:
import sys
sys.path.append('..') # for import src

import os
import cloudpickle
import lzma
import pandas as pd
import numpy as np
from scipy.stats import pearsonr
import matplotlib.pyplot as plt
import ccxt

import src
cloudpickle.register_pickle_by_value(src) # for model portability

In [11]:

from pytrends.request import TrendReq

class Fetcher:
    def __init__(self, keyword=None, geo=''):
        self.keyword = keyword
        self.geo = geo
        self.keys = {
            'keyword': keyword,
            'geo': geo,
        }
        self.data_id = 'google_trends'
        
    def fetch(self, last_timestamp=None):
        pytrend = TrendReq(hl='en', tz=0, retries=3, backoff_factor=0.1)
        
        if last_timestamp is None:
            end_time = pd.to_datetime('2013-10-06 21:00:00Z', utc=True)
        else:
            end_time = pd.to_datetime(last_timestamp + 60 * 60, unit='s', utc=True)
        start_time = end_time - pd.to_timedelta(24 * 7, unit='hour')

        timeframe = '{} {}'.format(
            start_time.strftime('%Y-%m-%dT%H'),
            end_time.strftime('%Y-%m-%dT%H'),
        )
        # timeframe='2020-01-01T00 2020-01-03T00'
        
        pytrend.build_payload(kw_list=[self.keyword], timeframe=timeframe, geo=self.geo)
        df = pytrend.interest_over_time()
        # display(df)
        
        df = df.reset_index()
        df = df.rename(columns={ 'date': 'timestamp', self.keyword: 'value' })
        df['timestamp'] = df['timestamp'].astype('int') // 10 ** 9

        df = df.loc[~df['isPartial']]
        df = df.iloc[-1:]
        df = df.set_index('timestamp')
        df = df[['value']].astype('float')
        
        return df


In [12]:
fetcher = Fetcher(keyword='btc')
df = fetcher.fetch(last_timestamp=None)
display(df)
df = fetcher.fetch(last_timestamp=(1600000000 // 3600) * 3600)
display(df)

,value
timestamp,
1381093200,0


,value
timestamp,
1600002000,61


In [13]:
symbols = 'BTC,ETH,XRP,LINK,ATOM,DOT,SOL,BNB,MATIC,ADA'.split(',')
symbols += ['ALGO', 'AVAX', 'BCH', 'APT']
symbols += 'DOGE,SFP,DYDX,AXS,CHZ,TRX,MASK,ETC,LTC,1000SHIB,C98,SAND,SUSHI,NEAR,FIL'.split(',')
fetchers = []

for symbol in symbols:
    fetchers.append(Fetcher(keyword=symbol.lower()))
    
data = cloudpickle.dumps(fetchers)
data = lzma.compress(data)
with open('/home/jovyan/data/20221127_google_trends.xz', 'wb') as f:
    f.write(data)